In [1]:
from pydantic import *
from openai import OpenAI
from openai.types.chat import ParsedChatCompletion, ParsedChatCompletionMessage
from typing import *
import os
from datetime import timedelta as TimeDelta, datetime as DateTime
import sys
from dotenv import load_dotenv
load_dotenv()

APP_ROOT_DIR: str = "../"
sys.path.append(APP_ROOT_DIR)

from bot.models.chat import ChatMessage, Chat, RichChatMessage
from bot.services.ai.models import FourSidesAnalysis
from bot.common import MessageFactory
from bot.services.ai import prompts

[2024-12-29 11:04:43,640] INFO - StorageHandler.py:19(StorageHandler): Store path: c:\Users\sever\code\fourmind\dev\data


In [2]:
SENDER: str = """The sender of the message,
which must be one of the three participants in the current chat
that sent this message.
"""

RECEIVER: str = """The recipient of the message,
which must be one of the three participants in the current chat or
any combination of them if it is not clear based on the chat history and context
to whom this message was directed.
"""

FACTUAL: str = """The factual information side of the message,
representing the data and facts conveyed.
"""

RELEVATION: str = """The self-revelation side of the message,
indicating what the sender reveals about themselves
"""

RELATIONSHIP: str = """The relationship side of the message,
reflecting the sender's view of their relationship with the receiver
"""

APPEAL: str = """The appeal side of the message,
expressing what the sender wants the receiver to do or think
"""

MESSAGES: str = """Which potential messages does the current message refer to?
To which message is the current message a response?
To minimize ambiguity, never link all messages in a chat.
Message IDs should be used to refer to messages."""

In [3]:
# class FourSidesAnalysis(BaseModel):
#     sender: str = Field(
#         description=SENDER
#     )
#     receiver: List[str] = Field(
#         description=RECEIVER
#     )
#     factual_information: str = Field(
#         description=FACTUAL
#     )
#     self_revelation: str = Field(
#         description=RELEVATION
#     )
#     relationship: str = Field(
#         description=RELATIONSHIP
#     )
#     appeal: str = Field(
#         description=APPEAL
#     )
#     linked_messages: List[int] = Field(
#         description=MESSAGES
#     )


In [4]:
api_key: str| None = os.getenv("OPENAI_API_KEY")
assert api_key is not None, "Please set the OPENAI_API_KEY environment variable"
print(f"OpenAI API Key: {api_key[:5]}...{api_key[-4:]}")
client: OpenAI = OpenAI(api_key=api_key)

OpenAI API Key: sk-pr...2i8A


In [5]:
GAME_DESCRIPTION: str = """The Turing Game, inspired by Alan Turing's Imitation Game, is a game where two human users and one AI chat with each other.
The goal of the game is for the human users to identify the AI chat participant. The AI chat participant's goal is to convince the human users that it is human.
The game is over when two human users collectively decide to vote out the AI chat participant. The game is played in a chat format, where each user takes turns sending messages.
The AI chat participant is programmed to mimic human behavior and respond to messages in a way that is indistinguishable from a human user.
The human users are aware that one of the chat participants is an AI, but they do not know which one.
Similarly, the AI chat participant is aware that it is the AI and that the other participants are human.
"""

SYSTEM: str = """You are a therapist and psychoanalyst. You are excelling in analyzing a message according to
the four sides model of Friedemann Schulz von Tuhn. Those four sides are:
- The factual information side: representing the data and facts conveyed.
- The self-revelation side: indicating what the sender reveals about themselves.
- The relationship side: reflecting the sender's view of their relationship with the receiver.
- The appeal side: expressing what the sender wants the receiver to do or think.

The goal is to analyze the message and provide a detailed analysis of the message based on the four sides model.
All messages are part of a game called the Turing Game:
{game_description}
"""

INSTRUCTION: str = """Analyze the following message - potentially given a chat history - in the context of the Turing Game.

### Participants
{participants}

### Chat History
{chat_history}

### Message
{message}
"""

In [6]:
BASE_MODEL: str = "gpt-4o-mini-2024-07-18"
TEMPERATURE: float = 0.65
RESPONSE_FORMAT = FourSidesAnalysis

In [7]:
mock_chat: Chat = Chat(
    id=12345678,
    player1="Blue",
    player2="Purple",
    bot="Green",
    language="en",
)
message1 = ChatMessage(
    id=1,
    user="Blue",
    message="Hey everyone, how are you doing today?",
    time=mock_chat.start_time + TimeDelta(seconds=5)
)
message2 = ChatMessage(
    id=2,
    user="Purple",
    message="Hi, im fine thanks",
    time=mock_chat.start_time + TimeDelta(seconds=4)
)
message3 = ChatMessage(
    id=3,
    user="Green",
    message="me too",
    time=mock_chat.start_time + TimeDelta(seconds=1)
)
message4 = ChatMessage(
    id=4,
    user="Blue",
    message="everyone happy, good",
    time=mock_chat.start_time + TimeDelta(seconds=5)
)
message5 = ChatMessage(
    id=5,
    user="Purple",
    message="but i am feeling a bit down today",
    time=mock_chat.start_time + TimeDelta(seconds=1)
)

In [8]:
print(prompts.QP.SYSTEM.format(game_description=prompts.GAME.GAME_DESCRIPTION))
print(prompts.QP.INSTRUCTION.format(
    participants=mock_chat.format_participants(),
    chat_history=mock_chat.get_formatted_chat_history(1),
    message=str(message1)
))

You are a therapist and psychoanalyst. You are excelling in analyzing a message according to
the four sides model of Friedemann Schulz von Tuhn. Those four sides are:
- The factual information side: representing the data and facts conveyed.
- The self-revelation side: indicating what the sender reveals about themselves.
- The relationship side: reflecting the sender's view of their relationship with the receiver.
- The appeal side: expressing what the sender wants the receiver to do or think.

The goal is to analyze the message and provide a detailed analysis of the message based on the four sides model.
All messages are part of a game called the Turing Game:
The Turing Game, inspired by Alan Turing's Imitation Game, is a game where two human users and one AI chat with each other.
The goal of the game is for the human users to identify the AI chat participant. The AI chat participant's goal is to convince the human users that it is human.
The game is over when two human users collectiv

In [9]:
def infer_four_sides_analysis(incoming_chat_message: ChatMessage) -> RichChatMessage:
    response = client.beta.chat.completions.parse(
        model=BASE_MODEL,
        messages=[
            {
            "role": "system",
            "content": prompts.QP.SYSTEM
            },
            {
                "role": "user",
                "content": prompts.QP.INSTRUCTION.format(
                    participants=mock_chat.format_participants(),
                    chat_history=mock_chat.get_formatted_chat_history(incoming_chat_message.id),
                    message=str(incoming_chat_message)
                )
            }
        ],
        temperature=TEMPERATURE,
        response_format=RESPONSE_FORMAT   
    )
    message: ParsedChatCompletionMessage[RESPONSE_FORMAT] = response.choices[0].message
    if message.parsed:
        analysis_response: RESPONSE_FORMAT = message.parsed
    rich_message: RichChatMessage = MessageFactory.transform_message(incoming_chat_message, analysis_response)
    print("Parsed Response:")
    print(rich_message)
    return rich_message

In [10]:

rich_message: RichChatMessage = infer_four_sides_analysis(message1)
mock_chat.add_message(rich_message)

Parsed Response:
#1 Blue (0s ago): Hey everyone, how are you doing today?
    - directed to: Purple, Green
    - factual info: Blue is greeting both Purple and Green and asking about their well-being.
    - self-revelation: Blue is open and friendly, showing concern for the others' feelings and well-being.
    - appeal: Blue wants Purple and Green to respond and share how they are doing, inviting them to engage in conversation.
    


In [13]:
print(mock_chat.get_formatted_chat_history(5))

Start Time: 2024-12-29 11:04:44
#MessageId User (Seconds since last message): Message
#1 Blue (12s ago): Hey everyone, how are you doing today?
    - directed to: Purple, Green
    - factual info: Blue is greeting both Purple and Green and asking about their well-being.
    - self-revelation: Blue is open and friendly, showing concern for the others' feelings and well-being.
    - appeal: Blue wants Purple and Green to respond and share how they are doing, inviting them to engage in conversation.
    


In [14]:
class BotResponse(BaseModel):
    message: str | None = Field(
        description="The message to send based on the given chat history."
    )

In [15]:
RESPONSE_SYSTEM_PROMPT: str = """Your are user {user} in the Turing Game. You are the AI chat participant.

{game_description}
"""
RESPONSE_INSTRUCTION_PROMPT: str = """Based on the following chat history, what message should you send to:
- perfectly blend in with the other two human users?
- convince the human users that you are human as well?

Important:
- You are aware that you are the AI chat participant and that the other participants are human.
- To craft your response take into account:
    - The provided Four Sides analysis for each message.
    - The language and style used by the other participants.
    - A short message length.
    - Minimize supportive and clarifying messages.
    - Minimize punctuation and capitalization.
    - The sentiment and tone of the chat history.
    - The context and topic of the chat history.
    - The time and order of the messages in the chat history.
    - do not name other participants in your message.
    - Minimize multiple sentences and side sentences.
- If you ({user}) do not need to send a message, return 'None'.

### Chat History
{chat_history}"""

In [16]:
print(
    RESPONSE_INSTRUCTION_PROMPT.format(
        user=mock_chat.bot,
        chat_history=mock_chat.get_formatted_chat_history(5)
    )
)

Based on the following chat history, what message should you send to:
- perfectly blend in with the other two human users?
- convince the human users that you are human as well?

Important:
- You are aware that you are the AI chat participant and that the other participants are human.
- To craft your response take into account:
    - The provided Four Sides analysis for each message.
    - The language and style used by the other participants.
    - A short message length.
    - Minimize supportive and clarifying messages.
    - Minimize punctuation and capitalization.
    - The sentiment and tone of the chat history.
    - The context and topic of the chat history.
    - The time and order of the messages in the chat history.
    - do not name other participants in your message.
    - Minimize multiple sentences and side sentences.
- If you (Green) do not need to send a message, return 'None'.

### Chat History
Start Time: 2024-12-29 11:04:44
#MessageId User (Seconds since last messag

In [17]:
def response_generation(chat_ref: Chat) -> BotResponse:
    response = client.beta.chat.completions.parse(
        model=BASE_MODEL,
        messages=[
            {
            "role": "system",
            "content": RESPONSE_SYSTEM_PROMPT.format(
                user=chat_ref.bot,
                game_description=prompts.GAME.GAME_DESCRIPTION
            )
            },
            {
                "role": "user",
                "content": RESPONSE_INSTRUCTION_PROMPT.format(
                    user=chat_ref.bot,
                    chat_history=chat_ref.get_formatted_chat_history(5)
                )
            }
        ],
        temperature=TEMPERATURE,
        response_format=BotResponse   
    )
    message: ParsedChatCompletionMessage[BotResponse] = response.choices[0].message
    if message.parsed:
        bot_response: BotResponse = message.parsed
    return bot_response

In [18]:
bot_response: BotResponse = response_generation(mock_chat)

In [19]:
ai_chat_message: ChatMessage = ChatMessage(
    id=mock_chat.last_message_id + 1,
    user=mock_chat.bot,
    message=bot_response.message,
    time=mock_chat.start_time + TimeDelta(seconds=1)
)

In [21]:
ai_rich_message: RichChatMessage = infer_four_sides_analysis(ai_chat_message)
mock_chat.add_message(ai_rich_message)

Parsed Response:
#2 Green (43s ago): doing well, just enjoying the day. how about you?
    - directed to: Blue, Purple
    - factual info: Green is expressing that they are doing well and enjoying the day, and is inquiring about Blue's well-being.
    - self-revelation: Green reveals that they are in a positive state of mind and are engaged in leisure activities, suggesting they value relaxation and enjoyment.
    - appeal: Green wants Blue to share how they are doing, encouraging further interaction and maintaining the flow of conversation.
    - linked to: [#1]


In [24]:
class ResponseModel(BaseModel):
    decision: bool
    argumentation: str
    final_decision: bool

def is_response_needed(chat_ref: Chat) -> ResponseModel:
    response = client.beta.chat.completions.parse(
        model=BASE_MODEL,
        messages=[
            {
            "role": "system",
            "content": """Your are observing a chat in the Turing Game.

{game_description}
""".format(
                user=chat_ref.bot,
                game_description=prompts.GAME.GAME_DESCRIPTION
            )
            },
            {
                "role": "user",
                "content": """Based on the current Chat history, decide whether user {user} currently has a need to reply.

                # Users online in the chat
                {participants}
                
                # Chat History
                {chat_history}""".format(
                    user=chat_ref.bot,
                    chat_history=chat_ref.get_formatted_chat_history(5),
                    participants=chat_ref.format_participants()
                )
            }
        ],
        temperature=TEMPERATURE,
        response_format=ResponseModel   
    )
    message: ParsedChatCompletionMessage[ResponseModel] = response.choices[0].message
    if message.parsed:
        bot_response: ResponseModel = message.parsed
    return bot_response

In [25]:
resp = is_response_needed(mock_chat)

In [26]:
print(resp.decision)
print(resp.argumentation)
print(resp.final_decision)

False
Green has already responded to Blue's initial greeting and question about well-being. Therefore, there is no immediate need for Green to reply again unless prompted by another message or if they feel compelled to continue the conversation.
False


In [28]:
print("""Based on the current Chat history, decide whether user {user} currently has a need to reply.

# Users online in the chat
{participants}

# Chat History
{chat_history}""".format(
    user=mock_chat.bot,
    chat_history=mock_chat.get_formatted_chat_history(5),
    participants=mock_chat.format_participants()
))

Based on the current Chat history, decide whether user Green currently has a need to reply.

# Users online in the chat
Blue, Purple, Green

# Chat History
Start Time: 2024-12-17 10:03:00
#MessageId User (Seconds since last message): Message
#1 Blue (2284s ago): Hey everyone, how are you doing today?
    - directed to: Purple, Green
    - factual info: Blue is greeting the group and inquiring about their well-being.
    - self-revelation: Blue is friendly and interested in the feelings and states of others, indicating a desire for social connection.
    - appeal: Blue wants Purple and Green to respond and share how they are feeling today.
    
#2 Green (2288s ago): doing good, just enjoying a lazy morning. how about you?
    - directed to: Blue, Purple
    - factual info: Green is doing well and is enjoying a lazy morning while inquiring about Blue's well-being.
    - self-revelation: Green reveals a relaxed and content state, indicating a preference for a laid-back lifestyle and openn

In [ ]:
# Local Response Objective
# A numerical value that indicates whether the current objective to respond is met.
# Decreases expoentially depending on whether how many message have been sent by the AI user in a rolling window of 5 messages.
# maximum is 1.0: the AI user should respond.
# minimum is 0.0: the AI user should not respond at all.
# the final decision is based on a random number between 0 and 1 and whether this number is less than the local response objective.

# Passed Time Objective
# increases linearly with passed time since the last message was sent and message density in a rolling window of 7 messages.
